In [1]:
from sklearn import linear_model
import numpy as np
import os
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
clf= linear_model.Lasso(alpha=0.1)

X = np.array([[0,0],[1,1],[2,2]])
Y = np.array([0,1,2])

clf.fit(X,Y)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [3]:
print(clf.coef_)
print(clf.intercept_)

[0.85 0.  ]
0.15000000000000002


In [4]:
# Now try it with my data!

# for now focus on a single point...

z=0
x=5  #(11 points in x dir)
y=4   #(78 points in y dir)


StepSize = 10 # how many output steps to predict over

In [5]:
DIR = '/data/oceans_output/open/racfur'
exp_list = ['4500yr_Windx0.50']
file_list =[]
for exp in exp_list:
    #filenames = os.listdir(os.path.join(DIR,exp))
    filenames = ['state_cat.nc']
    for file in filenames:
        file_list.append(os.path.join(DIR,exp,file))

print(file_list)

['/data/oceans_output/open/racfur/4500yr_Windx0.50/state_cat.nc']


In [ ]:
#print(file_list[0])
ds   = xr.open_dataset(file_list[0])
#print(ds)

In [ ]:
#Plot a timeseries of the various potential inputs 
# here plotting for a single grid cell. Plotting Temp for set grid point and 
# an 8 cell halo. Aim is to use this to predict Temp at a later time step

fig1 = plt.figure()

ds.Temp.isel(X=x-1,Y=y+1,Z=z).plot(label=('x-1,y+1'))
ds.Temp.isel(X=x  ,Y=y+1,Z=z).plot(label=('x  ,y+1'))
ds.Temp.isel(X=x+1,Y=y+1,Z=z).plot(label=('x+1,y+1'))
ds.Temp.isel(X=x-1,Y=y  ,Z=z).plot(label=('x-1,y  '))
ds.Temp.isel(X=x  ,Y=y  ,Z=z).plot(label=('x  ,y  '))
ds.Temp.isel(X=x+1,Y=y  ,Z=z).plot(label=('x+1,y  '))
ds.Temp.isel(X=x-1,Y=y-1,Z=z).plot(label=('x-1,y-1'))
ds.Temp.isel(X=x  ,Y=y-1,Z=z).plot(label=('x  ,y-1'))
ds.Temp.isel(X=x+1,Y=y-1,Z=z).plot(label=('x+1,y-1'))


fig1.legend()


In [ ]:
inputs = []
outputs = []

for time in range(0, len(ds.T.data)-StepSize, 50):
    for x in range(0,10,3):
        for y in range(0,77,3):
           inputs.append([ds.Temp.isel(T=time,X=x-1,Y=y+1,Z=z),
                          ds.Temp.isel(T=time,X=x  ,Y=y+1,Z=z),
                          ds.Temp.isel(T=time,X=x+1,Y=y+1,Z=z),
                          ds.Temp.isel(T=time,X=x-1,Y=y  ,Z=z),
                          ds.Temp.isel(T=time,X=x  ,Y=y  ,Z=z),
                          ds.Temp.isel(T=time,X=x+1,Y=y  ,Z=z),
                          ds.Temp.isel(T=time,X=x-1,Y=y-1,Z=z),
                          ds.Temp.isel(T=time,X=x  ,Y=y-1,Z=z),
                          ds.Temp.isel(T=time,X=x+1,Y=y-1,Z=z)])               
           outputs.append([ds.Temp.isel(T=time+StepSize,X=x,Y=y,Z=z)])
    
inputs=np.asarray(inputs)
outputs=np.asarray(outputs)



In [ ]:
print(inputs.shape)
print(outputs.shape)

#Split data into training and test data and then normalise (normalise all based on training data only)


In [ ]:
# Fit a model....

model= linear_model.Lasso(alpha=0.1)

model.fit(inputs,outputs)

In [ ]:
print(model.coef_)
print(model.intercept_)

In [ ]:
model.predict([[35, 35,35,35,35,35,35,35,35]])